# Tutorial 2: Index Construction - Solved

1. a) In your own words, describe the step-by-step process the Blocked Sort-Based Indexing (BSBI) algorithm uses to build an inverted index.
   b) In your own words, describe the step-by-step process the Single-Pass In-Memory Indexing (SPIMI) algorithm uses to build an inverted index.

The BSBI algorithm is an external sorting algorithm designed to build an index for collections that are too large to fit in memory. It works by breaking the problem into "blocks" that can fit in memory.The process is as follows:


i) Parse and Batch: The algorithm parses the document collection and accumulates (termID, docID) pairs in memory until it fills a block of a fixed, predefined size. Term to termID mapping is done using a dictionary and is created on the fly or can be pre-computed with a first pass through the block of documents.


ii) In-Memory Sort: It performs a fast in-memory sort on this single block of pairs, sorting them by termID (primary key) and docID (secondary key).


iii) Write to Disk: This sorted block (which is now a small, inverted index for one part of the collection) is written to a temporary file on disk.


iv) Repeat: The algorithm repeats steps 1-3, creating multiple sorted block files on disk (e.g., $f_1, f_2, \dots, f_n$) until the entire collection has been processed.Merge: In a final step, the algorithm opens all the temporary block files on disk and simultaneously merges them into one large, final inverted index. This merge is efficient because it only requires sequential reads from the sorted blocks.

The SPIMI algorithm also processes the collection in blocks, but it is more efficient and scalable.

The process is as follows:

i) Parse and create postings on the fly: SPIMI reads tokens (term, docID) one by one. It uses a hash table in memory as its dictionary. For each token, it looks up the term in the hash table and adds the docID directly to that term's postings list.

ii) Write to Disk: It continues this until it runs out of memory. At this point, it sorts the terms in its dictionary (a small sort) and writes the entire block (dictionary and postings lists) to a temporary file on disk.

iii) Repeat and Merge: It then frees its memory, creates a new empty dictionary, and starts processing the next block. Like BSBI, it finishes with a final merge of all sorted blocks.

2. State one advantage and one disadvantage of Blocked sort-based Indexing (BSBI) over Single-pass in-memory indexing (SPIMI). State one similarity between them.

Solution:

**Advantage of BSBI over SPIMI**: BSBI does not have much advantage over SPIMI though one minor advantage is that its memory management is simpler. It converts all terms to fixed-size termIDs (e.g., 4-byte integers). This makes the data being sorted (the (termID, docID) pairs) uniform and predictable in size. SPIMI, by contrast, must handle dynamically growing postings lists in memory

**Disadvantage of BSBI over SPIMI**: The two major disadvantages of BSBI are:Scalability: BSBI requires the entire term-to-termID dictionary (the global vocabulary) to fit in main memory. If the vocabulary is too large, the algorithm fails.Time Complexity: BSBI must perform an expensive sorting step on all (termID, docID) pairs in the collection. This gives it a computational complexity of $\Theta(T \log T)$. SPIMI avoids this massive sort, giving it a more efficient linear time complexity of $\Theta(T)$.

**Similarity:** Both BSBI and SPIMI process the collection by (1) segmenting it into blocks that can fit in memory, (2) creating an inverted index for each block, and (3) writing these intermediate inverted blocks to disk. Both algorithms conclude with a final merge step that combines all the sorted blocks into one large, final index.

3. State two issues faced with main and auxiliary indexes? In the context of websites, state one advantage of dynamic indexing?

Solution: 

Two issues with main and auxiliary indexes:

1. Costly Merges: Each time the small auxiliary index is merged into the large main index, it's an expensive operation. If the index is stored as one large file, this merge requires re-writing the entire main index, leading to an inefficient $\Theta(T^2/n)$ overall complexity.

2. Slower Query Performance:

During Search: Queries must be run across both the main index and the auxiliary index, and the results must be merged, which is slower than querying a single index.

During Merge: The merging process itself is resource-intensive and can temporarily slow down the entire search system.


**Advantage of dynamic indexing:** The key advantage is immediacy: Dynamic indexing allows new documents (like a new blog post, news story, or forum comment on a website) to be added to the index and become searchable almost immediately. This is a huge improvement over "periodic reconstruction," where the new content might not be searchable until the entire index is rebuilt from scratch, which could be hours or even a day later.

4. **Numerical:**  If we need $n log_2$ n comparisons (where $n$ is the number of termID-docID pairs) and $2$ disk seeks for each comparison, how much time would index construction for Reuters-RCV1 take if we used disk instead of memory for storage and an unoptimized sorting algorithm (i.e., not an external sorting algorithm)? Please show the steps and provide necessary explanation.

Assume, disk seek time  $= 5*10^{-3}$ seconds and $n$ for the Reuters-RCV1 corpus $= 100$ million $= 10^8$

Solution:

Our goal is to find the Total Time to build the index. The total time is the product of how many comparisons we must do and how much time each comparison costs.

$\text{Total Time} = (\text{Total Comparisons}) \times (\text{Time per Comparison})$


The problem states that an unoptimized sort requires $n \log_2 n$ comparisons, where $n$ is the number of `termID-docID` pairs.

* $n$: $100 \text{ million} = 10^8$
* $\log_2 n$: $\log_2(10^8)$
   * Using the logarithm power rule ($\log(x^y) = y \cdot \log(x)$), this becomes: $8 \times \log_2(10)$
   * Since $\log_2(10)$ is approximately $3.32$, this is: $8 \times 3.3219 \approx 26.575$
* Total Comparisons: $n \times \log_2 n = 10^8 \times 26.575$

A "naive" algorithm would try to compare items stored on disk, forcing the disk head to move for each comparison.

* The problem states each comparison requires 2 disk seeks (one seek to read the first item, one seek to read the second item).
* The time for one disk seek is $5 \times 10^{-3}$ seconds.
* Time per Comparison: $2 \text{ seeks} \times (5 \times 10^{-3} \text{ s/seek}) = 10 \times 10^{-3} \text{ s} = \mathbf{10^{-2} \text{ s}}$

* Total Time: $(\text{Total Comparisons}) \times (\text{Time per Comparison})$
* Total Time: $(10^8 \times 26.575) \times (10^{-2} \text{ s})$
* Total Time: $26.575 \times (10^8 \times 10^{-2}) \text{ s}$
* Total Time: $26.575 \times 10^6 \text{ seconds}$
* Total Time: 26,575,000 seconds

To put this number in context, 26,575,000 seconds is over 307 days
